# Main algorithmic components of the character extraction phase

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import math
import sys

In [ ]:
 ##show_image creates a normalized window to display the image and destroyes it after a random key

In [ ]:
def show_image(img, name):
    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
##img_shape returns the height and width of the image

In [ ]:
def img_shape(img):
    height, width = img.shape[0:2]
    return height, widt

In [ ]:
##edge_detection returns edges found with the Canny algorithm

In [ ]:
def edge_detection(gray_img):
    edges = cv2.Canny(gray_img, 170, 230, 5, L2gradient=False)
    return edges

In [ ]:
##compute_skew computes the rotation angle using the lines found with HoughLines transform

In [ ]:
def compute_skew(img):
    height, width = img_shape(img)
    img = cv2.GaussianBlur(img,(3,3),0)
    edges = edge_detection(img)
    lines = cv2.HoughLines(edges, 1, np.pi/180, 150)
    new_lines = []
    if not (lines is None):
        for i in range(len(lines)):
            for rho,theta in lines[i]:
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))
                cv2.line(img,(x1,y1),(x2,y2),(0, 255, 0),2)
                new_lines.append([[x1, y1, x2, y2]])
        angle = 0.0
        nb_lines = len(new_lines)
        for line in new_lines:
            angle += math.atan2(line[0][3]*1.0 - line[0][1]*1.0,line[0][2]*1.0 - line[0][0]*1.0)
        if(nb_lines > 0):
            angle /= nb_lines*1.0
    else:
        angle = 0
        
    return (angle * 180.0 / np.pi - 180) if (angle * 180.0 / np.pi ) > 90 else (angle * 180.0 / np.pi )

In [ ]:
##deskew uses the rotation angle calculated in the previous step and applies correction using Affine transform

In [ ]:
def deskew(img,angle = 0):
    rows, cols = img_shape(img)
    diam = round(math.sqrt(rows**2 + cols **2))
    mat = np.ones((diam, diam), 'uint8')
    y_offset = round((diam - cols) / 2)
    x_offset = round((diam - rows) / 2)
    mat[x_offset:x_offset + rows, y_offset:y_offset + cols] = img
    non_zero_pixels = cv2.findNonZero(mat)
    center, wh, theta = cv2.minAreaRect(non_zero_pixels)
    root_mat = cv2.getRotationMatrix2D(center, angle, 1)
    rotated = cv2.warpAffine(mat, root_mat, (diam, diam), flags=cv2.INTER_LINEAR)
    sizex = np.int0(wh[0])
    sizey = np.int0(wh[1])
    if theta > -45 :
        temp = sizex
        sizex= sizey
        sizey= temp
    return cv2.getRectSubPix(rotated, (sizey,sizex), center)

In [ ]:
##threshold uses Otsu algorithm to binarize the image, and then inverts it 

In [ ]:
def threshold(img):
    img = cv2.GaussianBlur(img,(3,3),0)
    ret,bin_img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return bin_img

In [ ]:
##equal_roi_delta creates a new black roi and using roi defenition coordinates performs the last stage of character processing

In [ ]:
def equal_roi_delta(lm, rm, um, dm, delta, mask):
    if(dm - um > rm - lm):
        if((dm - um - rm + lm)%2 == 0):
            diff = int((dm - um - rm + lm) / 2)
            roi = np.zeros(((dm-um + 2 * delta), (rm - lm + 2 * delta + 2 * diff)))
            roi[delta : delta + dm - um, diff + delta : diff + delta + rm - lm] = mask[um : dm,lm : rm]
        else:
            diff = int((dm - um - rm + lm) / 2)
            roi = np.zeros(((dm-um + 2 * delta), (rm - lm + 2 * delta + 2 * diff + 1)))
            roi[delta : delta + dm - um, diff + delta + 1 : diff + delta + 1 + rm - lm] = mask[um : dm,lm : rm]
    else:
        if((dm - um - rm + lm)%2 == 0):
            diff = int((rm - lm - dm + um) / 2)
            roi = np.zeros(((dm-um + 2 * delta + 2 * diff), (rm - lm + 2 * delta)))
            roi[delta + diff + 1: delta + diff + 1 + dm - um, delta: delta + rm - lm] = mask[um : dm,lm : rm]
        else:
            diff = int((rm - lm - dm + um) / 2)
            roi = np.zeros(((dm-um + 2 * delta + 2 * diff + 1), (rm - lm + 2 * delta)))
            roi[delta + diff + 1 : delta + diff + 1 + dm - um, delta: delta + rm - lm] = mask[um : dm,lm : rm]
    return roi

In [ ]:
##connected_components labels the different components in the binarized and deskewed image

In [ ]:
def connected_components(bin_img, filename):
    num_labels, labels_im = cv2.connectedComponents(bin_img)
    label_hue = np.uint8(179 * labels_im/np.max(labels_im))
    blank_ch = 255 * np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
    labeled_img[label_hue == 0] = 0
    count = 1
    comp_dict = {}
    lm_list = []
    for label in range(2,num_labels):
        mask = np.array(labels_im, dtype=np.uint8)
        mask[labels_im == label] = 255
        mask[labels_im != label] = 0
        surface = bin_img.shape[0] * bin_img.shape[1]
        if(np.sum(mask == 255) > int(0.001 * surface) and np.sum(mask == 255) < int(0.09 * surface)):
            white_pixels = np.array(np.where(mask == 255))
            um = white_pixels[:,0]
            dm = white_pixels[:,-1]
            white_pixels_T = np.array(np.where(mask.T == 255))
            lm = white_pixels_T[:,0]
            rm = white_pixels_T[:,-1]
            delta = 5
            if((float(dm[0] - um[0]) / float(rm[0] - lm[0])) >= 1):
                roi = equal_roi_delta(lm[0], rm[0], um[0], dm[0], delta, mask)
                if(np.sum(roi) != 0 and roi.shape[0] == roi.shape[1]):
                    resized_roi = cv2.resize(roi, (28, 28))
                    comp_dict.update({str(lm[0]) : resized_roi})
                    lm_list.append(lm[0])
    sort = sorted(lm_list)
    for value in sort:
        cv.imwrite("path", comp_dict[str(value)])